In [62]:
import together
from dotenv import load_dotenv
import os
import time
import pickle

### Fintune

In [91]:
load_dotenv()
together.api_key = os.getenv('TOGETHER_API_KEY')

In [92]:
resp = together.Files.check(file="cipher_data.jsonl")
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.004 GB', 'num_samples': 16000}


In [93]:
resp = together.Files.upload(file="cipher_data.jsonl")
print(resp)

Uploading cipher_data.jsonl: 100%|██████████| 3.85M/3.85M [00:03<00:00, 1.08MB/s]

{'filename': 'cipher_data.jsonl', 'id': 'file-34bd3c80-4d79-4149-a13d-5a93fd2d6043', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.004 GB', 'num_samples': 16000}}


In [96]:
fine_tune_resp = together.Finetune.create(
  training_file = resp['id'],
  model = 'togethercomputer/Llama-2-7B-32K-Instruct',
  n_epochs = 4,
  batch_size = 128,
  # lora = True, #super important
  suffix = 'cs224n-milestone-finetune',
  wandb_api_key = os.getenv('WAND_DB')
)

In [97]:
fine_tune_resp

{'training_file': 'file-34bd3c80-4d79-4149-a13d-5a93fd2d6043',
 'validation_file': '',
 'model_output_name': 'macvincentaghaoko@gmail.com/Llama-2-7B-32K-Instruct-cs224n-milestone-finetune-2024-03-02-02-13-20',
 'model_output_path': 's3://together-dev/finetune/65df9e44f5850c89099e8634/macvincentaghaoko@gmail.com/Llama-2-7B-32K-Instruct-cs224n-milestone-finetune-2024-03-02-02-13-20/ft-b718bd3e-22e9-4dd9-b8f7-2079b8f9f368',
 'Suffix': 'cs224n-milestone-finetune',
 'model': 'togethercomputer/Llama-2-7B-32K-Instruct',
 'n_epochs': 4,
 'n_checkpoints': 1,
 'batch_size': 4,
 'learning_rate': 1e-05,
 'eval_steps': 0,
 'user_id': '65df9e44f5850c89099e8634',
 'lora': False,
 'lora_r': 8,
 'lora_alpha': 8,
 'lora_dropout': 0,
 'staring_epoch': 0,
 'training_offset': 0,
 'checkspoint_path': '',
 'random_seed': '',
 'created_at': '2024-03-02T02:13:20.004Z',
 'updated_at': '2024-03-02T02:13:20.004Z',
 'status': 'pending',
 'owner_address': '0xdfcc9c9ecdd66266dfc6d8fc11d2b8eb0622e840',
 'id': 'ft-b71

In [113]:

fine_tune_id = fine_tune_resp['id']
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed, compressing
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False

completed
True


In [112]:
username = 'amacvincent'
wandb_url = f'https://wandb.ai/{username}/together?workspace=user-{username}'
print(wandb_url)

https://wandb.ai/amacvincent/together?workspace=user-amacvincent


### Inference

In [ ]:
{"text": "### Instruction:\n        Use the input below to create a deciphered response. \n\n        ### Input:\n        as for something gnidnatstuo ... it 's not here .\n\n        ### Response:\n\n        as for something outstanding ... it 's not here . [END]"}


In [114]:
model_name = fine_tune_resp["model_output_name"]
together.Models.start(model_name)

{'success': True,
 'value': '0666301fa30477b81cd62d7188261550f041c3ddda65d38bcd698231f3748c32-f738fe2a15a4c0fdc4697fc3bba1ff58379dedf524a2a57deca66f4c0525eed0'}

In [115]:
fine_tune_resp["model_output_name"]

'macvincentaghaoko@gmail.com/Llama-2-7B-32K-Instruct-cs224n-milestone-finetune-2024-03-02-02-13-20'

In [120]:
output = together.Complete.create(
  prompt = "### Instruction:\n        Use the input below to create a deciphered response. \n\n        ### Input:\n        as for something gnidnatstuo ... it 's not here .\n\n        ### Response:\n\n        ", 
  model = model_name,
  temperature = 0.2,
  top_k = 1,
  top_p = 1,
  repetition_penalty = 1.1,
  stop = ['[END]']
)
print(output)

{'status': 'finished', 'prompt': ["### Instruction:\n        Use the input below to create a deciphered response. \n\n        ### Input:\n        as for something gnidnatstuo ... it 's not here .\n\n        ### Response:\n\n        "], 'model': 'macvincentaghaoko@gmail.com/Llama-2-7B-32K-Instruct-cs224n-milestone-finetune-2024-03-02-02-13-20', 'model_owner': '', 'tags': {}, 'num_returns': 1, 'args': {'model': 'macvincentaghaoko@gmail.com/Llama-2-7B-32K-Instruct-cs224n-milestone-finetune-2024-03-02-02-13-20', 'prompt': "### Instruction:\n        Use the input below to create a deciphered response. \n\n        ### Input:\n        as for something gnidnatstuo ... it 's not here .\n\n        ### Response:\n\n        ", 'top_p': 1, 'top_k': 1, 'temperature': 0.2, 'max_tokens': 128, 'stop': ['[END]'], 'repetition_penalty': 1.1, 'logprobs': None, 'safety_model': None}, 'subjobs': [], 'output': {'choices': [{'text': "as for something outstanding ... it's not here . [END]"}], 'request_id': '85d

In [132]:
output['output']['choices'][0]['text']

"as for something outstanding ... it's not here . [END]"

In [122]:
with open('test_dataset.pickle', 'rb') as handle:
    test_dataset = pickle.load(handle)
test_input, test_output = test_dataset

In [123]:
print(len(test_input), len(test_output))

4000 4000


In [151]:
test_dataset_result = []
from tqdm import  tqdm

In [152]:
for i, input_prompt in tqdm(enumerate(test_input)):
    output = together.Complete.create(
        prompt = input_prompt,
        model = model_name,
        temperature = 0.2,
        top_k = 1,
        top_p = 1,
        repetition_penalty = 1.1,
        stop = ['[END]']
    )
    response = output['output']['choices'][0]['text']
    test_dataset_result.append([response, test_output[i]])
    if i%100 == 0:
        with open('test_result.pickle', 'wb') as handle:
            pickle.dump(test_dataset_result, handle)
    time.sleep(1)

578it [17:18,  1.80s/it]


HTTPError: 502 Server Error: Bad Gateway for url: https://api.together.xyz/api/inference

In [153]:
with open("test_result.pickle", 'rb') as f:
    original_vocab = pickle.load(f)

In [154]:
original_vocab

[['itmap that day and the roof was well protected . [END]',
  'it <unk> that day and the roof was well protected . [END]'],
 ['the new roof was installed quickly , which received impressive comments from two neighbors . [END]',
  'the new roof was installed quickly , which received impressive comments from two neighbors . [END]'],
 ['ild highly recommend your company . [END]',
  'i would highly recommend your company . [END]'],
 ['음식 네트포 는 그 프리즈가 고 낮은 것이 아니라 . [END]',
  "food often is n't that fresh and prices are high . [END]"],
 ['all three employees looked right at me and ignored me . [END]',
  'all three employees looked right at me and ignored me . [END]'],
 ['你会得到美味的饭店特点在面包和肉档乔。 [END]',
  'you get similar treatment at the meat and bakery counter . [END]'],
 ['only the sushi people treat the customers respectfully . [END]',
  'only the sushi people treat the customers respectfully . [END]'],
 ['this place is top notch , both locally . [END]',
  'this place is top notch , both loca